In [75]:
import csv
import json
import unicodedata
import numpy as np
import networkx as nx

import requests
import pandas as pd
from pattern import web
import matplotlib.pyplot as plt
import re

from matplotlib import rcParams
from bs4 import BeautifulSoup

In [76]:
def n_mean_help(alist,k_num):
    n_means=[]
    sum_k=sum(alist[:k_num])
    n_means.append(sum_k)
    for ith in range(k_num,len(alist)):
        sum_k=alist[ith]+sum_k
        sum_k=-alist[ith-k_num]+sum_k
        n_means.append(sum_k)
    return n_means

In [77]:
def n_mean(df,colname,k_num):
    response_vals=list(df[colname])
    n_means_res=n_mean_help(response_vals,k_num)
    lasts=[None]*(k_num)
    n_means_res.extend(lasts)
    n_means_res.pop(0)
    return n_means_res

In [84]:
def clean_data(file_name):
    games_data=pd.read_csv(file_name,index_col=False)

    del games_data['Unnamed: 0']

    games_data=games_data[(games_data['goals_all'] >= 0) & (games_data['assists_all'] >= 0)]
    
    predicted_assists=n_mean(games_data,'assists_all',5)
    predicted_goals=n_mean(games_data,'goals_all',5)


    games_data['response_assists']=predicted_assists
    games_data['response_goal']=predicted_goals
    games_data['response_points']=games_data['response_goal']+games_data['response_assists']


    games_data=games_data.sort(['draft_years', 'Drafts','Year','ith_game'])
    games_data=games_data.reset_index(drop=True)

    games_data['gm_result']=games_data['gm_result'].replace(['W'], 1)
    games_data['gm_result']=games_data['gm_result'].replace(['L'], 0)

    games_data['gm_loc']=games_data['gm_loc'].replace(['Home'], 1)
    games_data['gm_loc']=games_data['gm_loc'].replace(['Away'], 0)

    
    games_data['fc_win'].fillna(0, inplace=True)
    games_data['fc_loss'].fillna(0, inplace=True)
    games_data['fc_win']
    frames = []
    for ith in range(0,len(games_data)-10):
#     for ith in range(0,300):
        grouped_head=games_data.iloc[ith:(10+ith)]

        ShotHand=grouped_head.iloc[0]['ShotHand']
        Position=grouped_head.iloc[0]['Position']
        Url=grouped_head.iloc[0]['Url']
        Name=grouped_head.iloc[0]['Name']

        Height=grouped_head.iloc[0]['Height']
        Weight=grouped_head.iloc[0]['Weight']
        Drafts=grouped_head.iloc[0]['Drafts']
        draft_years=grouped_head.iloc[0]['draft_years']
        
        
        
        Response_Assists=grouped_head.iloc[0]['response_assists']
        Response_Goal=grouped_head.iloc[0]['response_goal']
        Response_Points=grouped_head.iloc[0]['response_points']
        Year=grouped_head.iloc[0]['Year']
        
        Year=grouped_head.iloc[0]['Year']


        mean_row=grouped_head[['gm_result','age','ith_game','gm_loc','goals_all','assists_all','plus_minus','pen_min','goals_ev',\
                    'goals_pp', 'goals_sh', 'assists_pp','assists_ev', 'assists_sh', 'num_shifts', 'time_on_ice',\
                   'hits_all', 'blocks_all', 'fc_win', 'fc_loss' ]].mean()
        new_df=mean_row.to_frame().T
        new_df['ShotHand']=ShotHand
        new_df['Position']=Position
        new_df['Url']=Url
        new_df['Name']=Name

        new_df['response_assists']=Response_Assists
        new_df['response_goal']=Response_Goal
        new_df['response_points']=Response_Points
        new_df['Year']=Year
        
        new_df['Height']=Height
        new_df['Weight']=Weight
        new_df['Drafts']=Drafts
        new_df['draft_years']=draft_years

        frames.append(new_df)

    df_frames=pd.concat(frames,ignore_index=True)
    new_df_frames=df_frames.groupby(['Url','Year'], as_index=False).apply(lambda x: x.iloc[:-5])
    new_df_frames=new_df_frames.reset_index(drop=True)

    return new_df_frames

In [85]:
def clean_year(year):
    cleaned_data=clean_data('Data/Game_Data/games_data_'+str(year)+'.csv')
    cleaned_data.to_csv('Data/Game_Data/clean_data/games_data_'+str(year)+'.csv')


In [90]:
for year in range(2015,2016):
    print year
    clean_year(year)


2015


/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [26]:
frames = []
for ith in range(0,300):
    grouped_head=grouped.iloc[ith:(10+ith)]
    
    ShotHand=grouped_head.iloc[0]['ShotHand']
    Position=grouped_head.iloc[0]['Position']
    Url=grouped_head.iloc[0]['Url']
    Name=grouped_head.iloc[0]['Name']
    
    Response_Assists=grouped_head.iloc[0]['response_assists']
    Response_Goal=grouped_head.iloc[0]['response_goal']
    Response_Points=grouped_head.iloc[0]['response_points']
    Year=grouped_head.iloc[0]['Year']
    

    mean_row=grouped_head[['gm_result','age','ith_game','gm_loc','goals_all','assists_all','plus_minus','pen_min','goals_ev',\
                'goals_pp', 'goals_sh', 'assists_pp','assists_ev', 'assists_sh', 'num_shifts', 'time_on_ice',\
               'hits_all', 'blocks_all', 'fc_win', 'fc_loss' ]].mean()
    new_df=mean_row.to_frame().T
    new_df['ShotHand']=ShotHand
    new_df['Position']=Position
    new_df['Url']=Url
    new_df['Name']=Name
    
    new_df['response_assists']=Response_Assists
    new_df['response_goal']=Response_Goal
    new_df['response_points']=Response_Points
    new_df['Year']=Year

    frames.append(new_df)
    
df_frames=pd.concat(frames,ignore_index=True)
new_df_frames=df_frames.groupby(['Url','Year'], as_index=False).apply(lambda x: x.iloc[:-5])
new_df_frames=new_df_frames.reset_index(drop=True)



In [27]:
new_df_frames

,gm_result,age,ith_game,gm_loc,goals_all,assists_all,plus_minus,pen_min,goals_ev,goals_pp,...,fc_win,fc_loss,ShotHand,Position,Url,Name,response_assists,response_goal,response_points,Year
0,0.4,20.074,5.5,0.6,0.6,0.5,0.1,0.8,0.4,0.2,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,3.0,2.0,5.0,2006.0
1,0.3,20.080,6.5,0.5,0.6,0.5,0.0,0.6,0.4,0.1,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,3.0,2.0,5.0,2006.0
2,0.3,20.088,7.5,0.4,0.6,0.4,0.0,0.6,0.4,0.1,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,2.0,4.0,6.0,2006.0
3,0.4,20.095,8.5,0.5,0.6,0.3,0.1,0.4,0.4,0.1,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,2.0,3.0,5.0,2006.0
4,0.4,20.103,9.5,0.5,0.7,0.3,-0.1,0.2,0.4,0.2,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,3.0,2.0,5.0,2006.0
5,0.4,20.110,10.5,0.5,0.8,0.3,-0.3,0.6,0.4,0.3,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,2.0,4.0,6.0,2006.0
6,0.4,20.118,11.5,0.5,0.9,0.2,-0.2,0.6,0.5,0.3,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,1.0,4.0,5.0,2006.0
7,0.3,20.125,12.5,0.4,0.9,0.1,-0.3,0.6,0.5,0.3,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,1.0,2.0,3.0,2006.0
8,0.4,20.132,13.5,0.5,0.8,0.1,-0.4,0.6,0.4,0.3,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,1.0,4.0,5.0,2006.0
9,0.4,20.139,14.5,0.4,0.9,0.2,-0.5,0.8,0.4,0.4,...,NaN,NaN,R,L,o/ovechal01,Alex Ovechkin,0.0,6.0,6.0,2006.0
